In [5]:
# Python Libraries
import random
import math
import numbers
import platform
import copy
import os
import time

# Importing essential libraries for basic image manipulations.
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import PIL

import torch
import torch.nn.functional as F
from torch import nn
from torch.utils.data import Dataset, DataLoader
from torchvision.io import read_image
import torchvision.transforms as transforms
import torchvision.transforms.functional as tF
import torchvision.models as models

In [ ]:
%matplotlib inline

# Enable/Disable GPU 
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [ ]:
class CustomImageDataset(Dataset):
    def __init__(self, annotations_file, img_dir, transform=None, test=False):
        img_labels = pd.read_csv(annotations_file, header = 0)
        if test:
            self.img_labels = img_labels.iloc[round(0.9 * img_labels.shape[0]):] 
        else:
            self.img_labels = img_labels.iloc[:round(0.9 * img_labels.shape[0])]
        self.img_dir = img_dir
        self.transform = transform

    def __len__(self):
        return len(self.img_labels)

    def __getitem__(self, idx):
        img_path = os.path.join(self.img_dir, self.img_labels.iloc[idx, 1])
        image = read_image(img_path)
        label = self.img_labels.iloc[idx, 2]
        if self.transform:
            image = self.transform(image)
        return image.float(), label

In [ ]:
batch_size = 4

transform = transforms.Compose(
    [transforms.RandomVerticalFlip(p=0.5),
     transforms.RandomHorizontalFlip(p=0.5)])

trainset = CustomImageDataset(annotations_file = './data/annotation_file.csv', 
                              img_dir='./data/champion-classifier', transform=transform, test=False)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, num_workers=0, shuffle=True)

testset = CustomImageDataset(annotations_file = './data/annotation_file.csv', 
                              img_dir='./data/champion-classifier', transform=None, test=True)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size, num_workers=0, shuffle=True)

In [7]:
embedding = nn.Embedding(10, 3)
# a batch of 2 samples of 4 indices each
input = torch.LongTensor([[1,2,4,5],[4,3,2,9]])
embedding(input)

tensor([[[-0.2953,  0.9859,  1.0614],
         [-1.5247, -0.5135, -2.0656],
         [-0.1733,  0.8146,  0.3970],
         [-0.1272,  0.3887, -0.3461]],

        [[-0.1733,  0.8146,  0.3970],
         [-2.0688, -0.6792,  0.1559],
         [-1.5247, -0.5135, -2.0656],
         [ 0.1893, -0.3965,  0.3569]]], grad_fn=<EmbeddingBackward0>)

### WIN PREDICTOR NET

In [12]:
ny, nx = 10, 10
x, y = np.arange(nx), np.arange(ny)
np.meshgrid(x, y)

[array([[0, 1, 2, 3, 4, 5, 6, 7, 8, 9],
        [0, 1, 2, 3, 4, 5, 6, 7, 8, 9],
        [0, 1, 2, 3, 4, 5, 6, 7, 8, 9],
        [0, 1, 2, 3, 4, 5, 6, 7, 8, 9],
        [0, 1, 2, 3, 4, 5, 6, 7, 8, 9],
        [0, 1, 2, 3, 4, 5, 6, 7, 8, 9],
        [0, 1, 2, 3, 4, 5, 6, 7, 8, 9],
        [0, 1, 2, 3, 4, 5, 6, 7, 8, 9],
        [0, 1, 2, 3, 4, 5, 6, 7, 8, 9],
        [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]]),
 array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [2, 2, 2, 2, 2, 2, 2, 2, 2, 2],
        [3, 3, 3, 3, 3, 3, 3, 3, 3, 3],
        [4, 4, 4, 4, 4, 4, 4, 4, 4, 4],
        [5, 5, 5, 5, 5, 5, 5, 5, 5, 5],
        [6, 6, 6, 6, 6, 6, 6, 6, 6, 6],
        [7, 7, 7, 7, 7, 7, 7, 7, 7, 7],
        [8, 8, 8, 8, 8, 8, 8, 8, 8, 8],
        [9, 9, 9, 9, 9, 9, 9, 9, 9, 9]])]

In [ ]:
# Function takes in cmp which is 2 dimensional 
# [ [10, 8],
#   [84, 3] ]
# Where i, j is the one hot representation of the champions on ith row, jth col
# trained_vec is a dictionary that convert each champions into a vector formate of size 2

def pretrain_init(cmp, champ2vec):
    
    def vectoring_champ(x):
        return champ2vec.get(x)
    
    return np.vectorize(vectoring_champ)(cmp)

In [ ]:
class Win_Predictor_Net(nn.Module):
    def __init__(self, criterion, 
                 cmp_size = 84, embedding_size = 4, hidden_size = 10):
        super(Win_Predictor_Net, self).__init__()

        self.embedding_size = embedding_size
        self.hidden_size = hidden_size
        # Embeddings to learn for champions 
        self.layer_cmp_emb = nn.Embedding(
            num_embeddings=self.cmp_size+1,
            embedding_dim=self.embedding_size,
            padding_idx=0)
        
        self.layer_w_0 = nn.Linear(
            in_features=self.embedding_size,
            out_features=self.hidden_size,
            bias=True)
        
        self.layer_w_1 = nn.Linear(
            in_features=self.embedding_size,
            out_features=1,
            bias=True)
        
        self.sigmoid = nn.sigmoid()
        
    def forward(self, scmp, ocmp, pretrain = False):

        # champion level embedings
        if not pretrain:
            E_self = self.layer_cmp_emb(scmp)
            E_opp = self.layer_cmp_emb(ocmp)
        else:
            E_self = pretrain_init(scmp)
            E_opp = pretrain_init(ocmp)
            
        E_self_flatten = E_self.view(-1, self.embedding_size)
        E_Tself_flatten = torch.tanh(self.layer_w_0(E_self_flatten))
        
        E_opp_flatten = E_opp.view(-1, self.embedding_size)
        E_Topp_flatten = torch.tanh(self.layer_w_0(E_opp_flatten))
        
        # concat
        E_concat = torch.cat((E_Tself_flatten, E_Topp_flatten), 1)
        E_concat_flatten = E_concat.view(-1, self.hidden_size)
        E_Tconcat_flatten = torch.tanh(self.layer_w_1(E_concat_flatten))

        return self.sigmoid(E_Tconcat_flatten)

In [ ]:
def train_model(model, dataloaders, optimizer, num_epochs=25):
    
    since = time.time()
    acc_list = []
    model.train() # In training mode

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        running_loss = 0.0
        running_corrects = 0

        # Iterate over data.
        for inputs, labels in dataloaders:
            inputs = inputs.to(device)
            labels = labels.to(device)

            # zero the parameter gradients
            optimizer.zero_grad()

            # forward
            outputs = model(inputs)
            loss = model.criterion(outputs, labels)

            _, preds = torch.max(outputs, 1)

            # backward + optimize only if in training phase
            loss.backward()
            optimizer.step()

            # statistics
            running_loss += loss.item() * inputs.size(0)
            running_corrects += torch.sum(preds == labels.data)

        # Epoch information
        epoch_loss = running_loss / len(dataloaders.dataset)
        epoch_acc = running_corrects.double() / len(dataloaders.dataset)
        acc_list.append(epoch_acc)

        print('Training Loss: {:.4f} Acc: {:.4f}'.format(epoch_loss, epoch_acc))


    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    
    return acc_list

In [ ]:
def eval_model(model, dataloaders):
    
    since = time.time()
    model.eval() # In training mode

    running_loss = 0.0
    running_corrects = 0

    # Iterate over data.
    for inputs, labels in dataloaders:
        inputs = inputs.to(device)
        labels = labels.to(device)
        
        with torch.no_grad():

            # forward
            outputs = model(inputs)
            loss = model.criterion(outputs, labels)

            _, preds = torch.max(outputs, 1)

            # statistics
            running_loss += loss.item() * inputs.size(0)
            running_corrects += torch.sum(preds == labels.data)

    overall_loss = running_loss / len(dataloaders.dataset)
    overall_acc = running_corrects.double() / len(dataloaders.dataset)

    print('Evaluation Loss: {:.4f} Acc: {:.4f}'.format(overall_loss, overall_acc))


    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    
    return overall_acc